<a href="https://colab.research.google.com/github/jason96819/Project_1/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5_KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

In [10]:
import pandas as pd
import numpy as np
import urllib.request
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt", filename="steam.txt")

('steam.txt', <http.client.HTTPMessage at 0x786356e3c2b0>)

In [5]:
total_data = pd.read_table('steam.txt', names=['label', 'reviews'])
print('전체 리뷰 개수 :',len(total_data))

전체 리뷰 개수 : 100000


In [6]:
total_data['reviews'].nunique(), total_data['label'].nunique()                                                                          # 중복확인

(99892, 2)

In [7]:
total_data.drop_duplicates(subset=['reviews'], inplace=True)                                                                        # 중복 샘플 제거
print('총 샘플의 수 :',len(total_data))

총 샘플의 수 : 99892


In [8]:
total_data.isnull().sum()                                                                                                                               # null 확인

label      0
reviews    0
dtype: int64

In [11]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 77)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

훈련용 리뷰의 개수 : 74919
테스트용 리뷰의 개수 : 24973


In [12]:
train_data.head()

,label,reviews
17424,0,어려워..
35511,0,g29 인식이 아주 쌉버러지입니다 아세토 아이레이싱은 인식 잘되는데 이 게임만 핸들...
85195,0,시발 애미뒤진게임 존나 은빨이고 파이어볼인데 위에 지상에 있는에 못죽이고 가드 선택...
92540,0,내취향은 아니다 왜샀는지 모르겠음 시네마틱 컷씬은 너무 많고 둠같은 스타일 기대했는...
49897,1,곰탱이 반사 침묵과 서큐버스 즉사 기술이 갈수록 사기적이지만 저는 인간남케 Adam...


In [13]:
test_data.head()

,label,reviews
51370,0,스페셜은 지원되는모드가 너무적네여 미리 알았으면 레전드리 샀는데 앞으로 사실 분들도...
93843,0,"인내심 유발 게임... 난이도 쉬움도 실제로는 어려운 편이라, 도전과제를 목표로 하..."
43272,1,"모놀리스의 AVP2, 리벨리온의 AVP3보다 리벨리온의 첫 AVP1이 공포감이 장난..."
43814,0,이게임 뭐하는지 모르겠다.
5439,0,마지막에 웨스커 바주카쏘고 터뜨린담에 쉐바 개 개좆같은년이 주사기를 쳐 안꽂아서 진...


In [14]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
max_length = max(len(tokens) for tokens in train_data['reviews'])
max_length

140

In [16]:
max_length = max(len(tokens) for tokens in test_data['reviews'])
max_length

140

In [17]:
max_seq_len = 140

In [19]:
print(tokenizer.encode("이게임 뭐하는지 모르겠다."))

[9018, 40766, 46651, 27481, 42138, 9016]


In [20]:
print(tokenizer.tokenize("모놀리스의 AVP2, 리벨리온의 AVP3보다 리벨리온의 첫 AVP1이 공포감이 장난 아닌 게임이다. 다만 게임이 옛날 게임이라 마린의 속도가 에일리언과 프레데터랑 별반 다를게 없다."))

['▁모', '놀', '리스의', '▁A', 'V', 'P', '2,', '▁리', '벨', '리', '온의', '▁A', 'V', 'P', '3', '보다', '▁리', '벨', '리', '온의', '▁첫', '▁A', 'V', 'P', '1', '이', '▁공포', '감이', '▁장난', '▁아닌', '▁게임', '이다.', '▁다만', '▁게', '임이', '▁옛날', '▁게임', '이라', '▁마', '린의', '▁속도가', '▁에', '일리', '언', '과', '▁프레데', '터', '랑', '▁별', '반', '▁다를', '게', '▁없다.']


In [21]:
tokenizer.decode(tokenizer.encode("모놀리스의 AVP2, 리벨리온의 AVP3보다 리벨리온의 첫 AVP1이 공포감이 장난 아닌 게임이다. 다만 게임이 옛날 게임이라 마린의 속도가 에일리언과 프레데터랑 별반 다를게 없다."))

'모놀리스의 AVP2, 리벨리온의 AVP3보다 리벨리온의 첫 AVP1이 공포감이 장난 아닌 게임이다. 다만 게임이 옛날 게임이라 마린의 속도가 에일리언과 프레데터랑 별반 다를게 없다.'

In [23]:
for elem in tokenizer.encode("이게임 뭐하는지 모르겠다."):
  print(tokenizer.decode(elem))

이
게임
뭐
하는지
모르겠
다.


In [25]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [26]:
train_X, train_y = convert_examples_to_features(train_data['reviews'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 74919/74919 [00:20<00:00, 3588.45it/s]


In [27]:
test_X, test_y = convert_examples_to_features(test_data['reviews'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 24973/24973 [00:05<00:00, 4323.38it/s]


In [28]:
# 최대 길이: 140
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [51200 20685  9705 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200]
각 인코딩의 길이 : 140
정수 인코딩 복원 : <|endoftext|> 어려워..<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endo

In [29]:
# 최대 길이: 140
input_id = train_X[1]
label = train_y[1]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [51200 15317 13201 29749 11445   739  7905 19977  8263  8153  7172  7182
 19724  8563 10607  9907  7897  8135 11446  9443 18981  9018 15403  7489
 28895  7285 29749 11445  9848  7397  9844  8084 11446  7489  9443 12577
 16576 21934  8550  7220 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200 51200 51200 51200 51200]
각 인코딩의 길이 : 140
정수 인코딩 복원 : <|endoftext|> g29 인식이 아주 쌉버러지입니다 아세토 아이레이싱은 인식 잘되는데 이 게임만 핸들 인식이 아주 버러지에요 인식만 잘되면 매일 했을텐데<|endoftext|><

In [30]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [31]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [32]:
outputs

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 140, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [33]:
outputs[0]

<KerasTensor: shape=(None, 140, 768) dtype=float32 (created by layer 'tfgpt2_model')>

In [34]:
outputs[1]

(<KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (created by layer 'tfgpt2_model')>,
 <KerasTensor: shape=(2, None, 12, 140, 64) dtype=float32 (c

In [35]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [36]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [37]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [38]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [40]:
model.fit(train_X, train_y, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
1873/1873 [==============================] - 261s 98ms/step - loss: 0.5453 - accuracy: 0.7246 - val_loss: 0.4640 - val_accuracy: 0.7841
Epoch 2/5
1873/1873 [==============================] - 173s 92ms/step - loss: 0.4217 - accuracy: 0.8078 - val_loss: 0.5222 - val_accuracy: 0.7650
Epoch 3/5
1873/1873 [==============================] - 173s 93ms/step - loss: 0.3544 - accuracy: 0.8442 - val_loss: 0.5407 - val_accuracy: 0.7792
Epoch 4/5
1873/1873 [==============================] - 173s 92ms/step - loss: 0.2885 - accuracy: 0.8774 - val_loss: 0.5159 - val_accuracy: 0.7774
Epoch 5/5
1873/1873 [==============================] - 173s 92ms/step - loss: 0.2298 - accuracy: 0.9066 - val_loss: 0.6819 - val_accuracy: 0.7672


In [41]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

25/25 [==============================] - 17s 469ms/step - loss: 0.6735 - accuracy: 0.7691
test loss, test acc:  [0.673541247844696, 0.7691106200218201]
